In [7]:
print("All okay!")

All okay!


In [8]:
import sys, os

print(os.getcwd())

project_root = os.path.abspath(os.path.join(os.getcwd(),'..'))

d:\UV Projects\agentic-research-assistant\notebook


In [10]:
project_root

'd:\\UV Projects\\agentic-research-assistant'

In [11]:
sys.path.append(project_root)

In [ ]:
from research_and_analyst.utils.model_loader import ModelLoader

In [ ]:
model_loader = ModelLoader()

model = model_loader.load_llm()

{"timestamp": "2026-01-01T10:15:30.152454Z", "level": "info", "event": "Initializing ApiKeyManager"}
{"timestamp": "2026-01-01T10:15:30.153538Z", "level": "warning", "event": "OPENAI_API_KEY is missing in environment variables"}
{"timestamp": "2026-01-01T10:15:30.153970Z", "level": "info", "event": "GOOGLE_API_KEY loaded successfully from environment"}
{"timestamp": "2026-01-01T10:15:30.155016Z", "level": "info", "event": "GROQ_API_KEY loaded successfully from environment"}
{"path": "D:\\UV Projects\\agentic-research-assistant\\research_and_analyst\\config\\configuration.yaml", "keys": [], "timestamp": "2026-01-01T10:15:30.155016Z", "level": "info", "event": "Configuration loaded successfully"}
{"config_keys": [], "timestamp": "2026-01-01T10:15:30.156582Z", "level": "info", "event": "YAML configuration loaded successfully"}


TypeError: ModelLoader.load_llm() missing 1 required positional argument: 'self'